In [8]:
# Import dependencies
import pandas as pd
import requests
import json
import os
import csv
import numpy as np
from pprint import pprint

# Import API key
from config import geoapify_key

Source state boundry data from an API https://public.opendatasoft.com/explore/dataset/us-state-boundaries/api/  

In [9]:
#source state boundry data from an API:
target_url = f"https://public.opendatasoft.com/api/records/1.0/search/?dataset=us-state-boundaries&q=&facet=name"
response = requests.get(target_url)
response_json = response.json()
response_json

{'nhits': 56,
 'parameters': {'dataset': 'us-state-boundaries',
  'rows': 10,
  'start': 0,
  'facet': ['name'],
  'format': 'json',
  'timezone': 'UTC'},
 'records': [{'datasetid': 'us-state-boundaries',
   'recordid': '426b88798c9440bf6d5a77795169831c11a6fd5d',
   'fields': {'centlat': 38.9047579,
    'intptlat': 38.9041031,
    'geo_point_2d': [38.9047679414, -77.0163213502],
    'intptlon': -77.017229,
    'gid': 20,
    'division': 5,
    'name': 'District of Columbia',
    'region': 3,
    'st_asgeojson': {'coordinates': [[[-77.1197509997839, 38.934349999852],
       [-77.040881999958, 38.9958450003095],
       [-76.9093930001818, 38.8928520002621],
       [-77.0390059996411, 38.7916449999142],
       [-77.0373429997596, 38.8146349998601],
       [-77.0411070000889, 38.822939999809],
       [-77.0387839998833, 38.8244730001069],
       [-77.0377550003866, 38.8301379997509],
       [-77.0418889997134, 38.8337099998371],
       [-77.041984000148, 38.8314840000691],
       [-77.0432

In [10]:
# save data from API in a json file 

output_file_path = "state_boundries.geojson"

with open(output_file_path, "w") as output_file:
    json.dump(response_json, output_file)

print(f"Data saved to {output_file_path}")

Data saved to state_boundries.geojson


ATTEMPT WITH GEOPIFY - PLEASE DO NOT ADD TO THE FINAL PROJECT

Add geopify API to state list --> Map Leaflet Set up --> https://www.geoapify.com/boundaries-api / We didn't use this approach because OpenDataSoft API had all necessary information and was easier to extract without a need to use data use in Geopify. 

In [2]:
# Target states - pull target list of states from csv file state_list_REFERENCE

# Import state list
state_list = pd.read_csv("Resources/state_list_REFERENCE.csv")

#create a list of states based on state column in state_list
state_list_address = state_list["state"].tolist()

In [3]:
# Create an empty list to store 'state' and 'place_id' pairs
state_place_ids = []

# Iterate through each state in the state_list
for state in state_list['state']:
    target_url = f"https://api.geoapify.com/v1/geocode/autocomplete?text={state}&limit=1&type=state&format=json&apiKey={geoapify_key}"
    response = requests.get(target_url).json()
    
    # Extract 'state' and 'place_id' from the JSON response
    if 'results' in response and response['results']:
        result = response['results'][0]
        state_name = result.get('state', '')
        place_id = result.get('place_id', '')
        
        # Append 'state' and 'place_id' to the list
        state_place_ids.append({'state': state_name, 'place_id': place_id})

# Print the list of 'state' and 'place_id' pairs
for pair in state_place_ids:
    print(pair)

{'state': 'Alaska', 'place_id': '51abd1ab01cab562c05906b243a18a1c5040f00101f9016e08110000000000c0020a'}
{'state': 'Alabama', 'place_id': '51831b841417b555c0593684190923a14040f00101f9019e78020000000000c0020a'}
{'state': 'Arkansas', 'place_id': '519d610f92aa1c57c059a1eaa1c7399a4140f00101f9016e77020000000000c0020a'}
{'state': 'Arizona', 'place_id': '51aeb6627fd9f05bc0591c5e10919a324140f00101f901e278020000000000c0020a'}
{'state': 'California', 'place_id': '51f1b73d4162b05dc059e6ecf88ac9594240f00101f9016386020000000000c0020a'}
{'state': 'Colorado', 'place_id': '516150a6d1e4665ac05959e59d9ed25c4340f00101f901a978020000000000c0020a'}
{'state': 'Connecticut', 'place_id': '512b2c5d66fd2e52c0590f9fcfdb33d34440f00101f901a287020000000000c0020a'}
{'state': 'District of Columbia', 'place_id': '51ddc7f2173c3f53c059e2ee079769724340f00101f9011579020000000000c0020a'}
{'state': 'Delaware', 'place_id': '51b2d6506aafd952c059b4ed0fef94584340f00101f9013e79020000000000c0020a'}
{'state': 'Florida', 'place_id': 

In [4]:
#create a dataframe from state_place_ids
state_place_ids_df = pd.DataFrame(state_place_ids)
duplicate_index = state_place_ids_df[state_place_ids_df['state'] == 'West Virginia'].index[1]

# Replace the data at the duplicate index with data for "Virginia" and the new place_id
state_place_ids_df.loc[duplicate_index, 'state'] = 'Virginia'
state_place_ids_df.loc[duplicate_index, 'place_id'] = '51f0e7f481a4aa53c0595f5d2f8763c14240f00101f9012a6b03000000000092030856697267696e6961'


#save in csv file
state_place_ids_df.to_csv('Resources/state_place_ids_REFERENCE.csv', index=False)

In [5]:
state_place_ids_df = pd.read_csv('Resources/state_place_ids_REFERENCE.csv')
#create a list of place ids based on state column in state_place_ids_df
place_id_list = state_place_ids_df["place_id"].tolist()
place_id_list

['51abd1ab01cab562c05906b243a18a1c5040f00101f9016e08110000000000c0020a',
 '51831b841417b555c0593684190923a14040f00101f9019e78020000000000c0020a',
 '519d610f92aa1c57c059a1eaa1c7399a4140f00101f9016e77020000000000c0020a',
 '51aeb6627fd9f05bc0591c5e10919a324140f00101f901e278020000000000c0020a',
 '51f1b73d4162b05dc059e6ecf88ac9594240f00101f9016386020000000000c0020a',
 '516150a6d1e4665ac05959e59d9ed25c4340f00101f901a978020000000000c0020a',
 '512b2c5d66fd2e52c0590f9fcfdb33d34440f00101f901a287020000000000c0020a',
 '51ddc7f2173c3f53c059e2ee079769724340f00101f9011579020000000000c0020a',
 '51b2d6506aafd952c059b4ed0fef94584340f00101f9013e79020000000000c0020a',
 '518d98d9e7b15d54c059ac336376bbc13b40f00101f9010279020000000000c0020a',
 '51ed56e07547c754c059a9b64027292a4040f00101f901a578020000000000c0020a',
 '519ebd3435b56d63c059d4d0066003983340f00101f901a38a020000000000c0020a',
 '515b40683dfc5357c059ec4bdb64f9f54440f00101f9017277020000000000c0020a',
 '514948a46dfc805cc059d85020a287d24540f00101f901447

In [11]:
#attempt to pull geojson data from geoapify api
place_id = '51802a6edc62e45ac05972bad16cc3954540f00101f901c778020000000000c0020a'
target_placeid_url = f"https://api.geoapify.com/v1/boundaries/consists-of?id={place_id}&geometry=geometry_1000&apiKey={geoapify_key}"
response = requests.get(target_placeid_url)
response_json = response.json()
response_json



{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'country': 'United States',
    'country_code': 'us',
    'state': 'Wyoming',
    'county': 'Laramie',
    'lon': -104.7268231,
    'lat': 41.2737321,
    'state_code': 'WY',
    'formatted': 'Laramie County, Laramie, Wyoming, United States of America',
    'address_line1': 'Laramie County',
    'address_line2': 'Laramie, Wyoming, United States of America',
    'name': 'Laramie County',
    'categories': ['administrative', 'administrative.county_level'],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database Licence',
     'url': 'https://www.openstreetmap.org/copyright',
     'raw': {'name': 'Laramie County',
      'type': 'boundary',
      'osm_id': 1411400,
      'name:en': 'Laramie County',
      'alt_name': 'Laramie',
      'boundary': 'administrative',
      'osm_type': 'r',
      'wikidata': 'Q483982',
      'wikipedia': 

Upon review of the example file the file goes all the way to the county level which is too detailed for the project at hand. Decided to look for an alternative source of the state boundry polygon coordinates. 

In [57]:
# Attempt at a loop storping data from the API --> DO NOT RUN
# Create an empty list to store response JSON data
polygon_state_coordinates = []

# Iterate through each place_id in the place_id_list
for place_id in place_id_list:
    target_placeid_url = f"https://api.geoapify.com/v1/boundaries/consists-of?id={place_id}&geometry=geometry_1000&apiKey={geoapify_key}"
    response = requests.get(target_placeid_url).json()
    
    # Extract and append the geometry coordinates
    if 'geometry' in response and 'coordinates' in response['geometry']:
        coordinates = response['geometry']['coordinates']
        polygon_state_coordinates.append(coordinates)

# Print the geometry coordinates for each place_id
for coordinates in polygon_state_coordinates:
    print(coordinates)

KeyboardInterrupt: 